In [10]:
using CRCBS
using JLD
using LightGraphs, MetaGraphs

In [13]:

function load_experiment_parameters(file)
    """Creates an experiment set with all the experiment sets"""
    experiment_vector = Vector{}()


    exp_set = load(string(file,".jld"))
    #println(exp_set)
    parameters = exp_set["parms"]

    #parameters = Experiment_parameters(experiment_vector)
    return parameters
end

function run_problem(name; sub_name="", lambda=1.0,epsilon=0.01,t_delay=1.0,dataframe=false,num_trials=1000)
    if sub_name == ""
        sub_name = name
    end
    exp_parms = load_experiment_parameters(sub_name)
    G = MetaGraph()
    for (k,v) in enumerate(exp_parms.vs)
        vnew = add_vertex!(G,Dict(:x=>exp_parms.xs[k],:y=>exp_parms.ys[k]))
    end
    for v in vertices(G)
        set_prop!(G, v,:n_delay, 1.0)
    end
    for e in exp_parms.es
        add_edge!(G,e[1],e[2])
        set_prop!(G, Edge(e[1],e[2]), :weight, 1.0)
    end
    mapf = MAPF(G,exp_parms.starts,exp_parms.goals)


    a = @timed(CBS(mapf))

    llsolution = a[1][1]
    cost = a[1][2]
    success = true
    if cost >=  typemax(Int)
        success = false
    end
    execution_time = a[2]

    countingtime = a[1][3]
    time_spent_on_astar = a[1][4]
    num_interactions = a[1][5][1]
    num_iterations = a[1][6]

    num_agents = length(exp_parms.starts)
    n_nodes = length(exp_parms.vs)
    n_edges = div(length(exp_parms.es),2)

    # Save solution
    solution_to_txt(llsolution, sub_name)

    # Save particle data ready for plotting

    solution_times = run_particles(mapf, llsolution,20) #20 particles
    jldopen(string("../experiments/simulations/",string(sub_name),".jld"),"w") do file
        write(file,"times",solution_times)
    end


    # ---------------------- Now make and fill dataframe --------------------- #
    if dataframe == false
        data = DataFrame(name=String[],
        Id=Int64[],
        CBS=Bool[],
        success=Bool[],

        #MAPF contents
        lambda=Float64[],
        epsilon=Float64[],
        t_delay=Float64[],

        # Experiment properties
        num_trials=Int64[],
        num_agents=Int64[],
        cbs_time_steps=Int64[],
        num_nodes=Int64[],
        num_edges=Int64[],
        #mapf::MAPF

        # Result storage
        solving_time=Float64[], #Time it took to find a solution
        counting_time=Float64[],
        time_spent_on_astar=Float64[],
        num_interactions=Int64[],
        num_iterations = Int64[],

        solution_cost=Float64[], #Cost of computer solution in ideal case
        global_cp=Float64[],
        conflict_counts_locally=Vector{Float64}[],
        probability_error=Tuple{Float64,Float64}[]) #Mean, std
    else
        data = dataframe
    end

    #Run particles for simulation
    solution_times = run_particles(mapf, llsolution,num_trials)

    # Get conflict stats
    (global_cp, conflict_counts_locally, mean_prob_err,std_prob_err) = get_conflict_stats(mapf,llsolution,solution_times,num_trials)
    probability_error = (mean_prob_err,std_prob_err)

    # Add it to the list of experiments from the set
    push!(data, [sub_name,0,false,success,mapf.lambda,mapf.epsilon,mapf.t_delay,num_trials,num_agents,-1,n_nodes, n_edges,execution_time,countingtime,time_spent_on_astar,num_interactions,num_iterations,cost,global_cp,conflict_counts_locally,probability_error])


    return data
end

run_problem (generic function with 1 method)

In [14]:
name = "Timing_test_10"
sub_name = "Timing_test_10_3"
data = run_problem(name; sub_name)

┌ Warning: type CRCBS.Experiment_parameters not present in workspace; reconstructing
└ @ JLD /home/peltzer/.julia/packages/JLD/jeGJb/src/jld_types.jl:697


Optimal Solution Found! Cost = 78


LoadError: BoundsError: attempt to access (Array{LightGraphs.SimpleGraphs.SimpleEdge{Int64},1}[[Edge 74 => 85, Edge 85 => 96, Edge 96 => 95, Edge 95 => 94, Edge 94 => 105, Edge 105 => 104, Edge 104 => 103, Edge 103 => 114], [Edge 3 => 14, Edge 14 => 25, Edge 25 => 26, Edge 26 => 37, Edge 37 => 48, Edge 48 => 49, Edge 49 => 50, Edge 50 => 61, Edge 61 => 62], [Edge 45 => 56, Edge 56 => 57, Edge 57 => 68, Edge 68 => 69, Edge 69 => 80, Edge 80 => 91, Edge 91 => 92, Edge 92 => 93, Edge 93 => 94, Edge 94 => 95, Edge 95 => 96], [Edge 91 => 80, Edge 80 => 69, Edge 69 => 58], [Edge 12 => 13, Edge 13 => 14, Edge 14 => 25, Edge 25 => 26, Edge 26 => 27, Edge 27 => 28, Edge 28 => 29, Edge 29 => 30, Edge 30 => 31, Edge 31 => 32, Edge 32 => 33, Edge 33 => 44], [Edge 86 => 75, Edge 75 => 74, Edge 74 => 63], [Edge 37 => 38, Edge 38 => 39, Edge 39 => 40, Edge 40 => 51], [Edge 27 => 38, Edge 38 => 49, Edge 49 => 60, Edge 60 => 61, Edge 61 => 72, Edge 72 => 73, Edge 73 => 84], [Edge 110 => 109, Edge 109 =